In [1]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

In [4]:
res = llm.invoke("Who is first walk on the moon?")

In [5]:
res.content

"The first walk on the moon was made by Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission."

In [14]:
from langchain_community.document_loaders import WebBaseLoader
url = 'https://careers.nike.com/accounting-admin-deer-park/job/R-64345'
loader = WebBaseLoader(url)
page_data = loader.load().pop().page_content

In [15]:
import re
page_data = re.sub("\n+", "\n", page_data)
page_data = re.sub(" +", " ", page_data).strip()

print(page_data)

Accounting Admin - Deer Park
Skip to main content
Open Virtual Assistant
Home
Career Areas
Total Rewards
Life@Nike
Purpose
Language
Select a Language
 Deutsch 
 English 
 Español (España) 
 Español (América Latina) 
 Français 
 Italiano 
 Nederlands 
 Polski 
 Tiếng Việt 
 Türkçe 
 简体中文 
 繁體中文 
 עִברִית 
 한국어 
 日本語 
Careers
Close Menu
Careers
Chat
 Home
 
 Career Areas
 
 Total Rewards
 
 Life@Nike
 
 Purpose
 
Jordan Careers
Converse Careers
Language
Menu
Return to Previous Menu
Select a Language
 Deutsch 
 English 
 Español (España) 
 Español (América Latina) 
 Français 
 Italiano 
 Nederlands 
 Polski 
 Tiếng Việt 
 Türkçe 
 简体中文 
 繁體中文 
 עִברִית 
 한국어 
 日本語 
 Back to Search
 
Accounting Admin - Deer Park
Categories ID
Categories URL
Position Type
Full Time
Date Posted
Primary Quest ID
Second Quest ID
Job Classification
Career area
Retail Stores
Location
102 Arches Cir, Deer Park, New York 11729, United States
Job ID
R-64345
 Apply Now
 
Share Job
Share Job Posting
Facebook
Opens In

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the
    following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm

In [16]:
res = chain_extract.invoke(input= {'page_data': page_data})

In [32]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()

result = json_parser.parse(res.content)
if type(result) == list:
    result = result[0]
print(result)

{'role': 'Accounting Admin - Deer Park', 'experience': '1 year of retail or consumer service experience', 'skills': ['Proficient knowledge of office practices, procedures, and equipment', 'Intermediate skills in Microsoft Office products including Word and Excel', 'Ability to learn and train on the latest products and technologies', 'Ability to communicate in English', 'Strong customer service skills and the ability to exercise a high degree of professionalism', 'Strong attention to detail and deadlines'], 'description': "As a Nike Retail Accounting Admin, you’re the face of NIKE. Enjoy high-volume and a fast pace as your diverse experience and perspective helps guide customers in making the best decisions for them. You’ll work with your team to focus on customer service and get to the win the right way. When we say team, we mean it. We go after goals together. We support your bold ideas — and encourage you to try them out. You impact our customers' experiences daily. It's more than ge

In [34]:
import pandas as pd
df = pd.read_csv('app/resource/my_portfolio.csv')
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [37]:
import chromadb
import uuid
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')
if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=row['Techstack'],
            metadatas={'link': row['Links']},
            ids=[str(uuid.uuid4())]
        )

In [41]:
collection.query(query_texts=["Expertise in React", 'Experience in python'], n_results=3)["metadatas"]

[[{'link': 'https://example.com/react-portfolio'},
  {'link': 'https://example.com/react-native-portfolio'},
  {'link': 'https://example.com/full-stack-js-portfolio'}],
 [{'link': 'https://example.com/ml-python-portfolio'},
  {'link': 'https://example.com/python-portfolio'},
  {'link': 'https://example.com/magento-portfolio'}]]